In [1]:
import pandas as pd
from pymongo import MongoClient, GEO2D
import pymongo
import json
import datetime
from bson.son import SON
import folium 
import math
import numpy as np
from IPython.display import display
import bus 

In [3]:
#Estableciendo la conexion con la base de datos morte_motoquiro en MongoDB
db = MongoClient("10.0.0.107", 27017).aniv_cidade_event

In [4]:
#Important: In geoWithin function, if you use longitude and latitude, specify longitude first.
#funcion que recibe un poligono y una coleccion de Mongo y te devuelve todos los puntos de la linea que aquel dia pasaron por aquel poligono
def points_inside_polygon(mongo_collection, inverted_polygon):
    points_inside_polygon = []
    for row in mongo_collection.find ({"latlon": {"$geoWithin": { "$polygon": inverted_polygon }}}).sort("timestamp",pymongo.ASCENDING):
        points_inside_polygon.insert(len(points_inside_polygon), [row[u'timestamp'], row[u'latitude'], row[u'longitude'], row[u'line'], row[u'bus_id']]) 
    df_points_inside_polygon = pd.DataFrame(points_inside_polygon, columns=['timestamp', 'latitude', 'longitude', 'line', 'bus_id'])
    return df_points_inside_polygon

In [4]:
#Poligono tramo 2-5
Punto2 = (-22.960298,-43.204346)
Punto5 = (-22.974325,-43.22625)
#inverted_polygon_2_5
T = (-22.974577,-43.226631)
inverted_T = (-43.226631,-22.974577)
V = (-22.974654,-43.225901)
inverted_V = (-43.225901,-22.974654)
W = (-22.961393,-43.215076)
inverted_W = (-43.215076,-22.961393)
X = (-22.961897,-43.214658)
inverted_X = (-43.214658,-22.961897)
Y = (-22.960128,-43.2041)
inverted_Y = (-43.2041,-22.960128)
Z = (-22.960484,-43.204015)
inverted_Z = (-43.204015,-22.960484)
inverted_polygon_2_5 = [inverted_V,inverted_T,inverted_W,inverted_Y,inverted_Z,inverted_X]

In [5]:
"""Funciones para renderizar el mapa en el html"""
from IPython.display import HTML
 
def inline_map(map):
    """
    Embeds the HTML source of the map directly into the IPython notebook.
    
    This method will not work if the map depends on any files (json data). Also this uses
    the HTML5 srcdoc attribute, which may not be supported in all browsers.
    """
    map._build_map()
    return HTML('<iframe srcdoc="{srcdoc}" style="width: 100%; height: 510px; border: none"></iframe>'.format(srcdoc=map.HTML.replace('"', '&quot;')))
 
def embed_map(map, path="map.html"):
    """
    Embeds a linked iframe to the map into the IPython notebook.
    
    Note: this method will not capture the source of the map into the notebook.
    This method should work for all maps (as long as they use relative urls).
    """
    map.create_map(path=path)
    return HTML('<iframe src="files/{path}" style="width: 100%; height: 510px; border: none"></iframe>'.format(path=path))

In [6]:
Polygon1 = [
    (-22.949205547938437, -43.1891655921936),
    (-22.950055195875997, -43.18897247314453),
    (-22.949857603808475, -43.18519592285156),
    (-22.949541455900263, -43.1823205947876),
    (-22.94871156412683, -43.182406425476074),
    (-22.949007954630193, -43.1856894493103),
    (-22.949205547938437, -43.1891655921936)
    ]


Polygon2 = [(-22.949699529946695, -43.19008827209473),
            (-22.949541455900263, -43.18950891494751),
            (-22.956615088692956, -43.186891078948975),
            (-22.956832429095073, -43.187384605407715),
           (-22.949699529946695, -43.19008827209473)]

Polygon3=[
            (-22.954639250838014, -43.18783521652222),
            (-22.955152971456624, -43.18755626678467),
            (-22.9570892854837, -43.19199800491333),
            (-22.958314286165276, -43.1956672668457),
            (-22.95890702444755, -43.19856405258179),
            (-22.958314286165276, -43.19882154464722),
            (-22.957682029134354, -43.19592475891113),
            (-22.956615088692956, -43.19227695465088),
            (-22.954639250838014, -43.18783521652222)]

Polygon4=[(-22.95772154528538, -43.1997013092041),
          (-22.958393318086337, -43.19897174835205),
          (-22.959697338116392, -43.20000171661377),
          (-22.96096183038056, -43.20300579071045),
          (-22.960922315176255, -43.20523738861084),
          (-22.961475526985673, -43.209614753723145),
          (-22.961910191819754, -43.21369171142578),
          (-22.96238437004533, -43.21480751037598),
          (-22.96716557423875, -43.21914196014404),
          (-22.966454331427162, -43.22017192840576),
          (-22.96171261706837, -43.215622901916504),
          (-22.960803769494103, -43.21420669555664),
          (-22.960290070337315, -43.21017265319824),
          (-22.959934431316487, -43.205881118774414),
          (-22.95957879136047, -43.20352077484131),
          (-22.95855138178991, -43.20068836212158),
         (-22.95772154528538, -43.1997013092041)]




# Create the map and add the line
m=folium.Map(location=[-22.95772154528538, -43.1997013092041], zoom_start=14)
m.line(Polygon1, line_color='#FF0000', line_weight=5)
m.line(Polygon2, line_color='blue', line_weight=5)
m.line(Polygon3, line_color='green', line_weight=5)
m.line(Polygon4, line_color='brown', line_weight=5)

embed_map(m)


In [7]:
def invert_polygon(polygon):
    polygon_inverted = []
    for points in polygon:
        polygon_inverted.insert(1, (points[1],points[0]))
    return polygon_inverted

In [8]:
Polygon1_inverted = invert_polygon(Polygon1)
Polygon2_inverted = invert_polygon(Polygon2)
Polygon3_inverted = invert_polygon(Polygon3)
Polygon4_inverted = invert_polygon(Polygon4)

In [15]:
points_inside_polygon1 = points_inside_polygon(db.dados20150301_line_410, Polygon1_inverted)
points_inside_polygon2 = points_inside_polygon(db.dados20150301_line_410, Polygon2_inverted)
points_inside_polygon3 = points_inside_polygon(db.dados20150301_line_410, Polygon3_inverted)
points_inside_polygon4 = points_inside_polygon(db.dados20150301_line_410, Polygon4_inverted)

In [19]:
for doc in points_inside_polygon4.iterrows():
    m.polygon_marker( [ doc[1]['latitude'], doc[1]['longitude'] ], fill_color='brown', num_sides=4, radius=6)
embed_map(m)

In [128]:
points_inside_polygon6['line'].unique()

array([u'315', u'352', u'360', u'382', u'613', u'2335', u'525', u'2329',
       u'LECD10', u'368', u'348', u'361', u'880', u'2333', u'2803',
       u'2338', u'332', u'2804', u'306', u'2334', u'614', u'887', u'2330',
       u'2115', u'863', u'612', u'692'], dtype=object)